#### Movielens 데이터를 활용해서 영화 추천 시스템 제작
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있다. MovieLens 1M Dataset 사용 권장
- 별점 데이터 - 대표적인 explicit 데이터이다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있다.
- 별점을 **시청횟수** 로 해석해서 생각
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외

## 1. 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있다.

In [1]:
import os
import pandas as pd

rating_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names = ratings_cols, engine='python', encoding='ISO-8859-1')
orginal_data_size = len(ratings)
ratings.head()


,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 영화 제목을 보기 위해 메타 데이터를 읽어온다.
movie_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# 검색의 용이성을 위해 영화 제목을 소문자로 바꿔준다.
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [4]:
# movie_id와 title 만 남겨준다
using_cols = ['movie_id', 'title']
movies = movies[using_cols]
movies.head()

,movie_id,title
0,1,toy story (1995)
1,2,jumanji (1995)
2,3,grumpier old men (1995)
3,4,waiting to exhale (1995)
4,5,father of the bride part ii (1995)


In [5]:
# ratings와 movies 데이터 프레임을 movie_id를 기준으로 병합
ratings = pd.merge(ratings, movies, on='movie_id')
ratings.head()

,user_id,movie_id,ratings,timestamp,title
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975)
1,2,1193,5,978298413,one flew over the cuckoo's nest (1975)
2,12,1193,4,978220179,one flew over the cuckoo's nest (1975)
3,15,1193,4,978199279,one flew over the cuckoo's nest (1975)
4,17,1193,5,978158471,one flew over the cuckoo's nest (1975)


In [6]:
# ratings 에서 timestamp 지우기
ratings.drop('timestamp', axis=1, inplace=True)
ratings.head()

,user_id,movie_id,ratings,title
0,1,1193,5,one flew over the cuckoo's nest (1975)
1,2,1193,5,one flew over the cuckoo's nest (1975)
2,12,1193,4,one flew over the cuckoo's nest (1975)
3,15,1193,4,one flew over the cuckoo's nest (1975)
4,17,1193,5,one flew over the cuckoo's nest (1975)


In [7]:
# 3점 이상만 남긴다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [8]:
# ratings 컬럼의 이름을 counts 로 바꾼다.
ratings.rename(columns={'ratings':'counts'}, inplace = True)

In [9]:
ratings['counts']

0          5
1          5
2          4
3          4
4          5
          ..
1000203    3
1000204    5
1000205    3
1000207    5
1000208    4
Name: counts, Length: 836478, dtype: int64

## 2. 분석해봅시다
- ratings에 있는 유니크한 영화 개수
- ratings 에 잇는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [10]:
ratings['movie_id'].nunique()

3628

In [11]:
ratings['title'].nunique()

3628

In [12]:
ratings['user_id'].nunique()

6039

In [13]:
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [16]:
# 유저별 몇 개의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

## 3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 준다

In [17]:
# 무슨 영화 있는지 확인
#set(movies['title'])

In [18]:
'''
# title-> movie_id 함수 만들기
def title_to_mid(title):
    m_id = []
    for i in title:
        mid = ratings[ratings['title'] == i]['movie_id']
        m_id.append(mid)
    return m_id
#movies[movies['title'] =='toy story (1995)']['movie_id']
my_favorite = ['clue (1985)', 'toy story (1995)', 'men in black (1997)', 'back to the future (1985)', 'jurassic park (1993)']
print(title_to_mid(my_favorite))
'''

"\n# title-> movie_id 함수 만들기\ndef title_to_mid(title):\n    m_id = []\n    for i in title:\n        mid = ratings[ratings['title'] == i]['movie_id']\n        m_id.append(mid)\n    return m_id\n#movies[movies['title'] =='toy story (1995)']['movie_id']\nmy_favorite = ['clue (1985)', 'toy story (1995)', 'men in black (1997)', 'back to the future (1985)', 'jurassic park (1993)']\nprint(title_to_mid(my_favorite))\n"

In [19]:
# 선호하는 영화 추가하기, + title-> movie_id 바꾼 값 my_movie에 넣어주기
my_favorite = ['clue (1985)', 'toy story (1995)', 'men in black (1997)', 'back to the future (1985)', 'jurassic park (1993)']

# 'aiffel' 이라는 유저가 위 영화의 평점을 5점씩 줬다고 가정
my_movie = pd.DataFrame({'user_id': ['aiffel']*5, 'title' : my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['aiffel']})['user_id'].any():
    ratings = ratings.append(my_movie)
    
ratings.tail(10)


,user_id,movie_id,counts,title
1000203,5556,2198.0,3,modulations (1998)
1000204,5949,2198.0,5,modulations (1998)
1000205,5675,2703.0,3,broken vessels (1998)
1000207,5851,3607.0,5,one little indian (1973)
1000208,5938,2909.0,4,"five wives, three secretaries and me (1998)"
0,aiffel,NaN,5,clue (1985)
1,aiffel,NaN,5,toy story (1995)
2,aiffel,NaN,5,men in black (1997)
3,aiffel,NaN,5,back to the future (1985)
4,aiffel,NaN,5,jurassic park (1993)


위에서도 확인했지만 movie_id 나 title 이나 통계값이 같다. 또한 내용도 겹치므로 movie_id나 title 중 하나를 drop 해준다.

In [20]:
ratings.drop('movie_id', axis=1, inplace=True)
ratings.tail(10)

,user_id,counts,title
1000203,5556,3,modulations (1998)
1000204,5949,5,modulations (1998)
1000205,5675,3,broken vessels (1998)
1000207,5851,5,one little indian (1973)
1000208,5938,4,"five wives, three secretaries and me (1998)"
0,aiffel,5,clue (1985)
1,aiffel,5,toy story (1995)
2,aiffel,5,men in black (1997)
3,aiffel,5,back to the future (1985)
4,aiffel,5,jurassic park (1993)


### 모델에 활용하기 위한 전처리

In [24]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
title_unique = ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드
user_to_idx = {v:k for k, v in enumerate(user_unique)}
title_to_idx = {v:k for k, v in enumerate(title_unique)}

In [26]:
len(user_unique)

6040

In [25]:
# 인덱싱 잘 되었나 확인
print(user_to_idx['aiffel'])
print(title_to_idx['toy story (1995)'])

6039
40


In [27]:
# 인덱싱을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 series 구해보고
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될테니 dropna()로 제거
temp_user_data = ratings['user_id'].map(user_to_idx).dropna()
if len(temp_user_data) == len(ratings):
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail..')
    
# title_to_idx로  title도 동일 방식으로 인덱싱
temp_title_data = ratings['title'].map(title_to_idx).dropna()
if len(temp_title_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_title_data
else:
    print('title column indexing Fail..')
    
ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,counts,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,1211
1,6039,5,40
2,6039,5,189
3,6039,5,22


## 4. CSR matrix를 직접 만들어보자

In [33]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_title = ratings['title'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape=(num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 본다

In [34]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분이다. 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [54]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model1 = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=90, dtype=np.float32)

In [55]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose를 해준다
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [56]:
# 모델 훈련
als_model1.fit(csr_data_transpose)

  0%|          | 0/90 [00:00<?, ?it/s]

## 6. 내가 선호하는 영화 중 하나와 그 외의 영화 골라서 훈련된 모델이 예측한 나의 선호도 파악해보기

In [61]:
aiffel, clue = user_to_idx['aiffel'], title_to_idx['toy story (1995)']
aiffel_vector, clue_vector = als_model1.user_factors[aiffel], als_model1.item_factors[clue]

In [62]:
aiffel_vector

array([ 0.32676134,  0.264871  , -0.20532419,  0.28868955,  0.02435688,
        0.24662629,  0.08869684, -0.2604811 , -0.11340661, -0.32359678,
        0.4743936 ,  0.43323216, -0.22028664,  0.35467514, -0.00151243,
       -0.42694944, -0.17896296, -0.07935438,  0.09488694,  0.00102733,
        0.05448223, -0.86994296,  0.26087606, -0.14897749, -0.11150809,
       -0.15115733,  0.49447447, -0.07763932, -0.02579144,  0.06689893,
       -0.08409509, -0.01845757, -0.36298028,  0.06617398,  0.36419588,
        0.04759718, -0.3298311 , -0.00372483, -0.3959943 , -0.11876844,
        0.28298494, -0.3893385 , -0.58818394,  0.19115351,  0.38156775,
       -0.3680207 ,  0.23635568, -0.5042326 ,  0.17356151, -0.12018202,
        0.45061284,  0.3297887 ,  0.29057458,  0.52308196, -0.00088042,
       -0.01194588,  0.21518241,  0.36851358,  0.17649804,  0.2636664 ,
        0.27497005, -0.46068203,  0.06717128,  0.22025253, -0.42732868,
        0.18123351,  0.21794222,  0.23890948,  0.30923858, -0.13

In [63]:
clue_vector

array([ 0.03511933,  0.05161886,  0.03924956,  0.05222552,  0.00373421,
        0.05292744,  0.00074268, -0.00370354, -0.01724284, -0.04932706,
        0.01973741,  0.01413761, -0.04136721, -0.02136711, -0.0275558 ,
       -0.00270693,  0.00388642, -0.00942425,  0.01337462,  0.00119051,
        0.04984034, -0.06960661,  0.026596  , -0.00360636, -0.00761764,
        0.03991732, -0.0065435 ,  0.02513026, -0.01218915,  0.04505363,
        0.03467635, -0.01057388, -0.01971217,  0.01108589,  0.0094981 ,
        0.02445908, -0.056647  ,  0.00726476, -0.00728418, -0.02334936,
       -0.02169806, -0.02572479, -0.03989544,  0.01140433, -0.00446312,
       -0.03404979,  0.05750902, -0.07470711,  0.00068131, -0.02879185,
        0.01756263,  0.03754517,  0.08276661,  0.0464677 , -0.0386342 ,
       -0.01262689,  0.03092838,  0.00357784, -0.00713709, -0.00882272,
       -0.01359763, -0.00193806, -0.00490084,  0.06402086, -0.06556155,
       -0.00452349,  0.04581225, -0.0296929 ,  0.02173186, -0.02

In [64]:
# 둘 내적해보기
np.dot(aiffel_vector, clue_vector)

0.4852326

수치가 심각하게 낮게 나왔다. factors나 iterations를 늘려본다.
iteration을 90으로 늘리고 다시 내적해도 영화 clue의 경우 내적한 값이 그대로 0.1정도로 낮았고 toy story는 0.48로 clue보다 높지만 여전히 낮은 수치이다. 선호하는 영화의 일관성이 없어서일까?

선호하는 영화 외의 영화 쥬만지를 예측해보자

In [65]:
ju = title_to_idx['jumanji (1995)']
ju_vector = als_model1.item_factors[ju]
np.dot(aiffel_vector, ju_vector)

-0.020473536

쥬만지는 내심 선호하는 축에 속하는데 마이너스 수치가 나왔다.\
영화 추천을 받아보자

## 7. 내가 좋아하는 영화와 비슷한 영화 추천받기

In [70]:
# 유저가 좋아하는 영화를 받아서, (영화id, 유사도) Tuple로 반환하고
# 영화 id를 다시 title로 매핑시켜주는 함수
idx_to_title = {v:k for k, v in title_to_idx.items()}

def get_similar_movie(title: str):
    title_id = title_to_idx[title]
    similar_movie = als_model1.similar_items(title_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [71]:
get_similar_movie('men in black (1997)')

['men in black (1997)',
 'jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'independence day (id4) (1996)',
 'matrix, the (1999)',
 'fifth element, the (1997)',
 'lost world: jurassic park, the (1997)',
 'face/off (1997)',
 'rocky horror picture show, the (1975)']

In [72]:
get_similar_movie('toy story (1995)')

['toy story (1995)',
 'toy story 2 (1999)',
 "bug's life, a (1998)",
 'aladdin (1992)',
 'babe (1995)',
 'groundhog day (1993)',
 'lion king, the (1994)',
 'pleasantville (1998)',
 'beauty and the beast (1991)',
 'forrest gump (1994)']

In [73]:
get_similar_movie('clue (1985)')

['clue (1985)',
 "'burbs, the (1989)",
 'haunted honeymoon (1986)',
 'funny farm (1988)',
 'ghostbusters ii (1989)',
 'one crazy summer (1986)',
 'police academy (1984)',
 'european vacation (1985)',
 'brighton beach memoirs (1986)',
 'little shop of horrors (1986)']

영화 추천을 보니 영화와 비슷한 장르의 영화를 뽑아주고 있다. 

## 8. 내가 가장 좋아할만한 영화 추천받기

In [74]:
user = user_to_idx['aiffel']

# recommend 에서는 user*item csr matrix를 받는다.
movie_recommended = als_model1.recommend(user, csr_data, N=10, filter_already_liked_items=True)
movie_recommended

[(97, 0.4693057),
 (50, 0.39946374),
 (132, 0.39224076),
 (4, 0.31467625),
 (92, 0.30785376),
 (171, 0.2973551),
 (160, 0.29382354),
 (62, 0.27553037),
 (733, 0.24199854),
 (33, 0.23949875)]

In [75]:
[idx_to_title[i[0]] for i in movie_recommended]

['terminator 2: judgment day (1991)',
 'toy story 2 (1999)',
 'matrix, the (1999)',
 "bug's life, a (1998)",
 'braveheart (1995)',
 'forrest gump (1994)',
 'independence day (id4) (1996)',
 'total recall (1990)',
 'galaxy quest (1999)',
 'aladdin (1992)']

추천받은 영화목록들을 찾아보니 내가 선호하는 영화와 장르가 거의 같은 류의 영화들이다. (브레이브하트 제외)

내가 가록을 남긴 데이터 중 이 추천(브레이브하트)에 기여한 정도를 확인해보자

In [76]:
bh = title_to_idx['braveheart (1995)']
explain = als_model1.explain(user, csr_data, itemid = bh)

In [77]:
explain

(0.3063068227943664,
 [(113, 0.15893833253385203),
  (22, 0.07373245885556676),
  (189, 0.056935507549786726),
  (1211, 0.04489739862395872),
  (40, -0.02819687476879783)],
 (array([[ 9.85925465e-01,  4.40256559e-02,  7.89622336e-02, ...,
           8.94219251e-02, -2.22627753e-03,  1.14365131e-01],
         [ 4.34060153e-02,  1.01396345e+00,  5.43312870e-02, ...,
           5.03943519e-02,  7.46889423e-02, -5.19342258e-02],
         [ 7.78508768e-02,  5.85663036e-02,  1.04338697e+00, ...,
           2.50493300e-02,  2.98797513e-02,  8.43407092e-02],
         ...,
         [ 8.81633531e-02,  5.50348901e-02,  3.59350896e-02, ...,
           9.15972722e-01, -2.04686475e-02,  1.07578142e-02],
         [-2.19494371e-03,  7.56338446e-02,  3.50582978e-02, ...,
           4.48277162e-04,  9.31898266e-01, -2.82074524e-02],
         [ 1.12755495e-01, -4.76244071e-02,  9.42088702e-02, ...,
           8.06091687e-02, -1.52876280e-02,  9.37459979e-01]]),
  False))

In [78]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('jurassic park (1993)', 0.15893833253385203),
 ('back to the future (1985)', 0.07373245885556676),
 ('men in black (1997)', 0.056935507549786726),
 ('clue (1985)', 0.04489739862395872),
 ('toy story (1995)', -0.02819687476879783)]

쥬라기 공원때문이었다. 쥬라기 공원이랑 브레이브하트랑 비슷한가? 

*** 
### 회고
비슷한 영화와 좋아할 만한 영화는 잘 추천받은 것 같다. 처음에 선호하는 영화를 내적해서 수치를 살펴볼 때는 그리 좋은 값이 나오지 않았지만, 결과적으로 잘 나온 것 같아 다행이다. 

중간에 선호하는 영화 5가지를 ratings에 추가해 줄 때 movie title로 추가해주었더니 movie_id는 신경쓰지 못해서 NaN 값이 나왔다. 이를 처리하기 위해 title을 movie_id로 바꿔주는 함수를 만드느라 시간을 허비했는데, 그냥 둘 중 하나만 남겨도 이상이 없을 데이터였다.. 그리고 함수도 못 만들었다. 역시 데이터를 분석하고 인공지능 공부하는 데에 기본적인 파이썬 실력은 필수구나 느꼈다. 특히 자료구조나 인덱싱, 슬라이싱을 더 공부해야겠다.